In [1]:
import re

import numpy as np
import pandas as pd
import os
import unicodedata


In [2]:
%cd "C:\Users\Administrador.PUESTO09\Desktop\Mapas_original\data\Sucia"

C:\Users\Administrador.PUESTO09\Desktop\Mapas_original\data\Sucia


### Tareas.xlsx -> Tareas-limpio.xlsx(Proyectos)

In [3]:
# 1. Carga inicial

df_tareas = pd.read_excel('Tareas.xlsx')

In [4]:
# 2. Verificación inicial de nulos

print("\n2. VERIFICACIÓN INICIAL DE NULOS")
print("------------------------------")

nulos_iniciales = df_tareas[['CODIGO', 'PROYECTO']].isna().sum()

for columna, cantidad in nulos_iniciales.items():
  print(f"- {columna}: {cantidad} nulos")


2. VERIFICACIÓN INICIAL DE NULOS
------------------------------
- CODIGO: 22 nulos
- PROYECTO: 0 nulos


In [5]:
df_tareas = df_tareas.dropna(subset=['CODIGO'])

In [7]:
# 4. Limpieza de PROYECTO

print("\n4. LIMPIEZA DE PROYECTO")
print("---------------------")

# Función para limpiar texto

def strip_accents(text):
  if pd.isna(text):
      return text
  return ''.join(c for c in unicodedata.normalize('NFD', str(text))
                if unicodedata.category(c) != 'Mn')

# Aplicar limpieza a PROYECTO

df_tareas['PROYECTO'] = df_tareas['PROYECTO'].apply(strip_accents)
print("✓ Acentos removidos de la columna PROYECTO")


4. LIMPIEZA DE PROYECTO
---------------------
✓ Acentos removidos de la columna PROYECTO


In [8]:
# 5. Selección final de columnas

print("\n5. SELECCIÓN DE COLUMNAS")
print("----------------------")

columnas_finales = ['CODIGO', 'PROYECTO']

df_tareas = df_tareas[columnas_finales]
print(f"Columnas seleccionadas: {', '.join(columnas_finales)}")



5. SELECCIÓN DE COLUMNAS
----------------------
Columnas seleccionadas: CODIGO, PROYECTO


In [9]:
# convertir CODIGO en int64

df_tareas['CODIGO'] = df_tareas['CODIGO'].astype('int64')


In [10]:
# 6. Guardado de archivo
print("\n6. GUARDANDO ARCHIVO LIMPIO")
print("-------------------------")
try:
  df_tareas.to_excel("../Limpia/Tareas-limpio.xlsx", index=False)
  print("✓ Archivo guardado exitosamente en '../Limpia/Tareas-limpio.xlsx'")
  
  # Verificación final
  print("\nEstado final del DataFrame:")
  print(f"- Filas: {df_tareas.shape[0]}")
  print(f"- Columnas: {df_tareas.shape[1]}")
  print("\nMuestra de datos limpios:")
  print(df_tareas.sample(min(5, len(df_tareas))))
  
  print("\nVerificación final de nulos:")
  nulos_finales = df_tareas.isnull().sum()
  for columna, cantidad in nulos_finales.items():
      print(f"- {columna}: {cantidad} nulos")
except Exception as e:
  print(f"❌ Error al guardar archivo: {e}")

print("\n=== FIN DE LIMPIEZA DE TAREAS ===")


6. GUARDANDO ARCHIVO LIMPIO
-------------------------
✓ Archivo guardado exitosamente en '../Limpia/Tareas-limpio.xlsx'

Estado final del DataFrame:
- Filas: 1841
- Columnas: 2

Muestra de datos limpios:
      CODIGO               PROYECTO
1239   62775  Ruta Alejandro Jueves
162    41935      Ruta Hugo Viernes
135    82185       Ruta Hugo Jueves
1304   80258  Ruta Alejandro Sabado
249    12894       Ruta Hugo Martes

Verificación final de nulos:
- CODIGO: 0 nulos
- PROYECTO: 0 nulos

=== FIN DE LIMPIEZA DE TAREAS ===


In [11]:
df_tareas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1841 entries, 0 to 1862
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODIGO    1841 non-null   int64 
 1   PROYECTO  1841 non-null   object
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [12]:
valores_unicos = df_tareas['PROYECTO'].unique()

# Imprimir los valores únicos
print(valores_unicos)

['Ruta Hugo Lunes' 'Ruta Hugo Martes' 'Ruta Hugo Miercoles'
 'Ruta Hugo Jueves' 'Ruta Hugo Viernes' 'Ruta Hugo Sabado'
 'Ruta Dario Lunes' 'Ruta Dario Martes' 'Ruta Dario Miercoles'
 'Ruta Dario Jueves' 'Ruta Dario Viernes' 'Ruta Dario Sabado'
 'Ruta Alejandro Lunes' 'Ruta Alejandro Martes' 'Ruta Alejandro Miercoles'
 'Ruta Alejandro Jueves' 'Ruta Alejandro Viernes' 'Ruta Alejandro Sabado'
 '🗞 Visitas periodicas Martin' '🗞 Visitas Periodicas Dario'
 '🗞 Visitas periodicas Alejandro' 'Clientes Especiales' '💲Creditos Hugo'
 '💲Creditos Dario' '💲Creditos Alejandro' '💲Creditos Administracion']
